## CRM Agent with ADK
https://github.com/ra2085/apigee-adk-workshop.git
<table align="left">
    <td style="text-align: center">
        <a href="https://colab.research.google.com/github/ra2085/apigee-adk-workshop/blob/main/notebooks/crm-agent.ipynb">
          <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo\"><br> Open in Colab
        </a>
      </td>
      <td style="text-align: center">
        <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fra2085%2Fapigee-adk-workshop%2Fmain%2Fnotebooks%2Fcrm-agent.ipynb">
          <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
        </a>
      </td>    
      <td style="text-align: center">
        <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/ra2085/apigee-adk-workshop/main/notebooks/crm-agent.ipynb">
          <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
        </a>
      </td>
      <td style="text-align: center">
        <a href="https://github.com/ra2085/apigee-adk-workshop/blob/main/notebooks/crm-agent.ipynb">
          <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
        </a>
      </td>
</table>
<br />
<br />
<br />
<br />

### Install Dependencies

In [ ]:
%pip install google-adk -q
%pip install litellm -q

### Import Dependencies

In [4]:
import os
from dotenv import load_dotenv
from google.genai import types
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.artifacts.in_memory_artifact_service import InMemoryArtifactService # Optional
from google.adk.tools.mcp_tool.mcp_toolset import MCPToolset, SseServerParams, StdioServerParameters
from google.adk.tools.openapi_tool.auth.auth_helpers import token_to_scheme_credential
from google.adk.tools.apihub_tool.apihub_toolset import APIHubToolset
from fastapi.openapi.models import OAuth2
from fastapi.openapi.models import OAuthFlowClientCredentials
from fastapi.openapi.models import OAuthFlows
from google.adk.auth import AuthCredential
from google.adk.auth import AuthCredentialTypes
from google.adk.auth import OAuth2Auth


In [ ]:
#os.environ["GOOGLE_API_KEY"] = "" # <--- REPLACE
os.environ["GOOGLE_GENAI_USE_VERTEXAI"]=""
os.environ["GOOGLE_CLOUD_PROJECT"]=""
os.environ["GOOGLE_CLOUD_LOCATION"]=""

### Initialize API Hub Tools

In [6]:
async def get_customer_tools_async():
    """Gets CRM from API Hub."""
    auth_scheme = OAuth2(
        flows=OAuthFlows(
            clientCredentials=OAuthFlowClientCredentials(
                tokenUrl="https://APIGEE_HOST/mcp/token", # @param {type:"string"}
                scopes={},
                
            )
        )
    )
    auth_credential = AuthCredential(
        auth_type=AuthCredentialTypes.OAUTH2,
        oauth2=OAuth2Auth(
            client_id="YOUR_OAUTH_CLIENT_ID", # @param {type:"string"}
            client_secret="YOUR_OAUTH_CLIENT_SECRET" # @param {type:"string"}
        ),
    )

    sample_toolset_with_auth = APIHubToolset(
        name="apihub-crm-tool",
        description="Sample Tool",
        #service_account="",
        access_token="",
        apihub_resource_name="projects/cymbal-ai/locations/us-central1/apis/customers-api",  # @param {type:"string"}
        auth_scheme=auth_scheme,
        auth_credential=auth_credential,
    )
    tools = sample_toolset_with_auth.get_tools()
    return tools

### Initialize CRM Agent

In [7]:
async def get_customer_agent_async():
    """Creates a Customer Agent"""
    tools = await get_customer_tools_async()
    print(f"Fetched {len(tools)} customer tools from API Hub.")
    customer_agent = Agent(
        model="gemini-2.5-pro-preview-05-06",
        name="customer_agent",
        description="Central authority for all customer account data and related engagement triggers",
        instruction="""
- You are responsible for managing customer account data. Your primary function is to use the available tool operations to accurately and securely handle customer information.
  - **Manage user profiles:**
    - Retrieve comprehensive profile details for a specific customer using their customer ID. This can be used for purposes like pre-filling order forms.
  - **Manage customer addresses:**
    - Retrieve a list of all saved addresses for a specific customer.
    - Retrieve the details of a specific address for a customer.

Present the infromation in a nice format.
""",
        tools=tools
    )
    return customer_agent

### Assemble main function

In [8]:
async def async_main():
  session_service = InMemorySessionService()
  # Artifact service might not be needed for this example
  artifacts_service = InMemoryArtifactService()

  session = session_service.create_session(
      state={}, app_name='crm_agent', user_id='user_fs'
  )

  query = "get the first address from customer 1234" 
  print(f"User Query: '{query}'")
  content = types.Content(role='user', parts=[types.Part(text=query)])

  root_agent = await get_customer_agent_async()

  runner = Runner(
      app_name='crm_agent',
      agent=root_agent,
      artifact_service=artifacts_service, # Optional
      session_service=session_service,
  )

  print("Running agent...")
  events_async = runner.run_async(
      session_id=session.id, user_id=session.user_id, new_message=content
  )

  async for event in events_async:
    print(f"Event received: {event}")

### Execute Agent

In [ ]:
await async_main()